In [1]:
import pandas as pd
import numpy as np

In [ ]:
# прочитаем train_2_col.csv
train_4_col_2 = pd.read_csv('train_4_col_2.csv', sep='\t', index_col = 0)
train_4_col_2_small = train_4_col_2.head(nrows)
train_4_col_2_small.head()

In [ ]:
# прочитаем train_4_col_3 .csv
train_4_col_3 = pd.read_csv('train_4_col_3.csv', sep='\t', index_col = 0)
train_4_col_3_small = train_4_col_3.head(nrows)
train_4_col_3_small.head()

In [ ]:
# прочитаем train_4_col.csv
train_4_col = pd.read_csv('train_4_col.csv', sep='\t', index_col = 0)
train_4_col_small = train_4_col.head(nrows)
train_4_col_small.head()

#### Объединим маленькие датафреймы

<font color="red" size="3">Внимание!!!.</font><br> 

чтобы объединились полные датафреймы - надо убрать окончание "_small" в ячейке ниже. Далее будет использовано название train. После объединения полных датафреймов это название менять не надо.

In [ ]:
train = train_4_col_3.merge(train_4_col_2, on='id', how='left').merge(train_4_col, on='id', how='left')
train.head(2).T

In [ ]:
train.info()

In [ ]:
# Функция, которая извлекает значений ключей из поля "fields"
def valextractor(field, key1=0, key2='name'):
    try:
        value = eval(field[1:-1])[key1]['field'][key2]         
    except SyntaxError:
        value = ''
    except KeyError:
        value = ''
    return value

In [ ]:
# проверяем функцию. работает.
field = train.iloc[0,1]
valextractor(field, key1=0,key2='name')

In [ ]:
# функция, которая доавляет новые категории
def getcat(df):
    df_ = df.copy()
    df_['fd_name1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='name')))
    df_['fd_name2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='name')))
    df_['fd_id1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='id')))
    df_['fd_id2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='id')))
    df_['fd_slug1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='slug')))
    df_['fd_slug2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='slug')))
    df_['fd_slug_id1'] = df_['fields'].map(lambda x: (valextractor(x, key1=0,key2='slug_id')))
    df_['fd_slug_id2'] = df_['fields'].map(lambda x: (valextractor(x, key1=1,key2='slug_id')))
    return df_

In [ ]:
# применим функцию
train = train.pipe(getcat)

In [ ]:
# удалим задублированные столбцы
train.drop(columns=['fd_id1','fd_id2','fields' ],axis=1, inplace=True)
# посмотрим что получилось
train.head(50).T

### Проведем морфологический анализ

In [ ]:
import re
import pymorphy2 # Морфологический анализатор.
#from collections import Counter 

In [ ]:
morph=pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь. теперь переменная morph может проводить морфологический анализ

In [ ]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)  # Хочу получить все значимые слова, но в уже нормальной форме
                break
    return words

In [ ]:
pd.set_option('max_rows',100)

Вариант 1. Создадим столбец 'morph_description' - где будет описание, прошедшее морфологический анализ. 

In [ ]:
# пропустим этот вариант
#%%time
#train['morph_description'] = train['description'].fillna('').astype('object')\
#.map(lambda text: ' '.join([morph.parse(r)[0].normal_form for r in re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)]))


In [ ]:
# посмотрим что получилось
#train.loc[5000,'morph_description']

Вариант 2. Посмотрим что получится при другом варианте морфологического анализа - тот же столбец 'morph_description'. 

In [ ]:
%%time
train['morph_description'] = train['description'].fillna('').astype('object')\
.map(lambda text: ' '.join(getMeaningfullWords(text)))

In [ ]:
train.loc[3,'morph_description']

Сделаем такой же с функцией **getMeaningfullWords** "морфологический текст" для столбца "name"

In [ ]:
# уже знакомая функция, но для названий: с английскими буквами и цифрами
def getMeaningfullWords_forname(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё0-9]+\-[А-Яа-яЁё0-9]+|[А-Яа-яЁё0-9]+|[A-Za-z0-9]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['NOUN' ]:           
                words.append(p.normal_form)  # Хочу получить все значимые слова, но в уже нормальной форме
            break
    return words

In [ ]:
%%time
train_['morph_name_2'] = train_['morph_name'].fillna('').astype('object')\
.map(lambda text: ' '.join(getMeaningfullWords_forname(text)))

In [ ]:
n = 9003
print('Название ДО морфологического анализа    :',train.loc[n,'name'])
print('Название ПОСЛЕ морфологического анализа :',train.loc[n,'morph_name'])

## Получим списки с частотами по Tf-Idf

In [ ]:
corpus = train['morph_description'] # Создадим корпус значимых слов
n = 3
example = train.loc[n,'morph_description'] # получаем n-ю строку

In [ ]:
tfCounter=TfidfVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
analyze = tfCounter.build_analyzer()
res=tfCounter.fit_transform(corpus) # Скармливаем сюда корпус значимых слов

In [ ]:
res2=analyze(example) # подаем ранее пролученную n-ю строку

# Получим значения частот всех слов n-й строки
tfs=list(set(res[n][0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys())) # укажем номер n 

# получим значения частот только тех слов, значения которых превышают среднее значение частот n-й строки
tfs2=[k for k in tfs if k>np.average(tfs)]

# или получим значения частот только тех слов, значения которых превышают среднее значение + Стандартное отклонение 
# частот n-й строки
#tfs2=[k for k in tfs if k>np.average(tfs)+np.std(tfs)]

print({w:res[n][0, tfCounter.vocabulary_[w]] for \
          w in res2 if w in tfCounter.vocabulary_.keys() and\
       res[n][0, tfCounter.vocabulary_[w]] in tfs2})


Или  вариант со средним значением

In [ ]:
n = 3
example = train.loc[n,'morph_description'] # получаем строку

res2=analyze(example) # подаем строку


tfs=list(set(res[n][0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys()))

#tfs2=[k for k in tfs if k>np.average(tfs)]
tfs2=[k for k in tfs if k>np.average(tfs)+np.std(tfs)]

print({w:res[n][0, tfCounter.vocabulary_[w]] for \
          w in res2 if w in tfCounter.vocabulary_.keys() and\
       res[n][0, tfCounter.vocabulary_[w]] in tfs2})


Возможно это еще пригодится. Вставить это в новый столбец, старое описание удалить, запустить dictVectorizer - , должно получиться покомпактнее, возможно будет и результат получше.

In [ ]:
# Просто почитать описания
row = 31
train.loc[row,'description']

In [ ]:
# Просто посмотреть список значимых слов
train_

### Сохраним, прочитаем train

In [ ]:
#train.to_csv('train_merged_full_morphed_2.csv')
train = pd.read_csv('train_merged_full_morphed_2.csv', index_col=0)
#train_.drop(columns=['fd_name1','fd_name2','fd_slug_id1','fd_slug_id2',\
#                    'fd_slug1','fd_slug2','description','name','Unnamed: 0' ],axis=1, inplace=True) #07.01.19


#train_['morph_description'] = train['morph_description'].fillna('').astype('object')
train['morph_name_2'] = train['morph_name_2'].fillna('').astype('object')

In [ ]:
pd.set_option('display.max_rows',500)

In [2]:
#train.to_csv('train_merged_full_morphed_2_light.csv')
train = pd.read_csv('train_merged_full_morphed_2_light.csv', index_col=0)

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
train.head(2)

,subcategory,morph_name_2
0,914,сумка
1,2202,комплект


### Поробуем Tf-Idf в "чистом" виде

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# Задаим стоп слова
stop_words1 = ['век','год','месяц','день','час','минута','секунда','иза','продать','купить','звонить','цена','влюбиться',
'состояние','дорг','размер','танк','продавать','сантиметр', 'брать','быть','пользование','понадобиться','любой',
'рубль','контакт','продаваться']
print('Количество стоп-слов составляет:',len(stop_words1))

Количество стоп-слов составляет: 27


In [7]:
# Задаим стоп слова для второго векторайзера
stop_words2 = [               
   'новое','цена','флисовыя','кожа','зам','набор','танец','кузнецов','мальчик', 'новое', 'новый', 'новая', 'метр', 'год', 'вт',  
  'дюйм', 'литр',  'рубль', 'дым', '100р' ,'28шт', 'мл', '1' ,'2','3', '4', '7', '8', '9', '10', '11', '12', '13', '14', '15', 
    '16', '17', '18', '19', 'rx', 'сутки', 'вренда', 'рубль', 'ps3', 'superslim', 's', 'торг',  'фламинго', 'g', '104плюс', 
    '110куб', '12-18мес','128г', '1970г', '1год', 'магаз', 'орига', '2007год', '2008г', '2008год', '200р', '2010г', '2016г', 
    '21-22р', '2-4г', 'двойнить', '3000игра',  '5500р', '5копейка',  'состояние', '7000-шт', '76см', '80см', '22-36кг', 'бронь',
    'стиль', 'сша', 'рост', 'обмен', 'литр','2003год', '2004г', '2003г', 'цвет', '62-64р', 'метр', '1500мма',
    '18мес', '21', '22', '22р', '36кг', '4г', '62', '64р', '7000', 'шт'
]
print('Количество стоп-слов составляет:',len(stop_words2))

Количество стоп-слов составляет: 97


In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words)
Tfidf_array = vectorizer.fit_transform(train['morph_description'])
#print('Список слов, определяющих частотный список:',vectorizer.get_feature_names())
print('Размеры (shape) полученного массива Tfidf_array (строк, слов):', Tfidf_array.shape)

In [ ]:
# Так можно получить матрицу частот - полежит пока здесь, на всякий случай..
#pd.set_option('display.max_rows', 500) 
#df_tfidf = pd.DataFrame(Tfidf_array.toarray())

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import LabelBinarizer

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [9]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

 Зададим селектор

In [10]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X_, y=None): # 
        return self
    def transform(self, X_):
        return X_.loc[:, self.attribute_names].values # 

In [11]:
class DenseTransformer(BaseEstimator,TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        #return X.todense()
        return X.toarray()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

Зададим списки столбцов

In [12]:
# получим список столбцов 
full_columns_list = list(train) 

# список числовых категориальных столбцов
num_cat_columns = ['subcategory']

tfidf_columns = ['morph_name_2']

# целвое значение
target = ['price']
print('num_cat_columns :',num_cat_columns)
print('tfidf_columns   :',tfidf_columns)

other = list(set(full_columns_list) - set(num_cat_columns)  - set(tfidf_columns) - set(target))
print('Остаток         :', other)

num_cat_columns : ['subcategory']
tfidf_columns   : ['morph_name_2']
Остаток         : []


<font color="blue" size="3">Pipline 1.</font><br> 

In [13]:
cat_pipeline = Pipeline([  
('selector_2', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['subcategory']
('OHE', OneHotEncoder())
])

In [14]:
# Это для проверки. Не запускать
model_categories = cat_pipeline.fit_transform
categories_array = model_categories(train)
df_categories_array = pd.DataFrame(categories_array.toarray())
df_categories_array.info()

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1748890 entries, 0 to 1748889
Columns: 250 entries, 0 to 249
dtypes: float64(250)
memory usage: 3.3 GB


In [ ]:
# Это для проверки. Не запускать
df_concat = pd.concat([train,df_categories_array], axis = 1)
df_concat.info()

<font color="blue" size="3">Pipline 2.</font><br> 

In [14]:
from sklearn.decomposition import PCA

In [15]:
tfidf_pipeline = Pipeline([  
('selector', ColumnSelector(tfidf_columns[0])), # определим столбцы для  tfidf  преобразования: ['morph_name_2']
('Tf*Idf', TfidfVectorizer(max_features = 4000,  stop_words=stop_words2)), #  max_features = 2000, max_df=0.95, min_df =0.05,
('dense',DenseTransformer()),
('p1' , PCA(n_components = 10)) 
])

In [ ]:
%%time
#train['morph_name_2'] = train['morph_name_2'].fillna('').astype('object')

tfidf_pipeline.fit(train.iloc[:500000,:]) # 

In [16]:
import pickle

In [ ]:
with open('tfidf_pipeline_10param.pickle', 'wb') as f:
     pickle.dump(tfidf_pipeline, f)

In [ ]:
with open('tfidf_pipeline.pickle', 'rb') as f:
     tfidf_pipeline = pickle.load(f)

In [ ]:
%%time
step = 5000
max_value = len(train) #.iloc[:100000,:]
cols = 10
X_pca_full_10params = np.zeros((0, cols))

for i in range(0,max_value,step):
    X_pca_i = tfidf_pipeline.transform(train.iloc[i:(i+step),:]) #.iloc[:500000,:]
    X_pca_full_10params = np.vstack((X_pca_full_10params, X_pca_i))

In [ ]:
X_pca_full_10params_df = pd.DataFrame(X_pca_full_10params)

In [17]:
# train_10params.to_csv('train_10params')
train_10params = pd.read_csv('train_10params.csv', index_col=0)
X = train_10params.drop(['category','price','morph_name_2'], axis = 1)
y = train_10params['price']

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
train_10params.head(2)

,category,subcategory,price,morph_name_2,0,1,2,3,4,5,6,7,8,9
0,9,914,1999.0,сумка,-0.049054,-0.048199,-0.008001,-0.111666,-0.213300,0.952322,0.018150,0.004751,0.069384,0.034727
1,22,2202,350.0,комплект,-0.036504,-0.019080,-0.003647,-0.019831,-0.014673,-0.018714,-0.006571,-0.002524,-0.026301,-0.023684


In [ ]:
with open('X_pca_full_10params.pickle', 'wb') as f:
     pickle.dump(X_pca_full_10params, f)

In [ ]:
with open('X_pca_full_p2.pickle', 'wb') as f:
     pickle.dump(X_pca_full, f)

</font><br>  <font color="magenta" size="5">CatBoost.</font><br> 


In [19]:
from catboost import CatBoostRegressor
catboost_model = CatBoostRegressor(depth = 6, num_trees=700, learning_rate = 0.11,cat_features=[0], verbose=False)

In [20]:
catboost_model.fit(X,y)

Got unsafe target value = 2.73e+06 at object #1380 of dataset learn


In [ ]:
catboost_model

In [ ]:
# CatBoost Вариант БЕЗ PCA
#model_2= Pipeline([('f', featureunion_pipeline), ('x', catbost_model)])

In [ ]:
from sklearn.metrics import mean_squared_log_error 

In [ ]:
model_2.get_params().keys()

In [ ]:
model_2.fit(X_train, y_train)

### Чистый CatBoost

In [ ]:
%%time 
catbost_model.fit(X_small, np.log(y_small+1))


In [ ]:
y_small.head(20).T

In [ ]:
import pickle
with open('catbost_model.pickle', 'wb') as f:
     pickle.dump(catbost_model, f)

In [ ]:
with open('catbost_model.pickle', 'rb') as f:
     catbost_model = pickle.load(f)

In [ ]:
X.drop(['0_x','1_x','2_x','3_x','0_y','1_y', '2_y', '3_y'], axis = 1, inplace=True)


In [ ]:
X_te = X.iloc[1000000:1700000,:]
y_te = y.iloc[1000000:1700000]
print(X_te.shape)
print(y_te.shape)

In [ ]:
y_hat_log = catbost_model.predict(X_te)
y_hat_log.shape

In [ ]:
y_hat = np.exp(y_hat_log)-1

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print(mean_absolute_error(y_te,y_hat))

In [ ]:
train__ = train_.iloc[1000000:1700000,:] # возьмем только предсказанную часть трейна
train__.shape

In [ ]:
y_te_df = pd.DataFrame(y_te)
y_hat_df = pd.DataFrame(y_hat)

In [ ]:
y_hat_df.index = range(1000000,1700000)

In [ ]:
# объединим предсказанную часть трейна, цену и предсказанную цену
train_predicted = pd.concat([train__, y_hat_df], axis=1)

In [ ]:
train_predicted.head(50)

In [ ]:
train_predicted.shape

In [ ]:
print(y_te.shape)
print(y_hat.shape)

# Загрузим и подготовим тестовые данные

In [21]:
test = pd.read_csv('test_merged_full_morphed.csv', index_col= 0)
#test.to_csv('test_merged_full_morphed.csv')
test['morph_name_2'] = test['morph_name_2'] .fillna('').astype('object')
test.head()

,subcategory,morph_name_2
0,603,стремянка
1,203,плита
2,116,диск
3,1009,
4,1104,ледобур рыбалка


In [ ]:
%%time
step = 5000
max_value = len(test) #.iloc[:100000,:]
cols = 10
test_pca_full_10params = np.zeros((0, cols))

for i in range(0,max_value,step):
    X_pca_i = tfidf_pipeline.transform(test.iloc[i:(i+step),:]) #.iloc[:500000,:]
    test_pca_full_10params = np.vstack((test_pca_full_10params, X_pca_i))

In [ ]:
#test_pca_full_10params_df = pd.DataFrame(test_pca_full_10params)
test_pca_full_10params_df.head()

In [ ]:
test = pd.read_csv('test_.csv', sep = ";", index_col=0)
#test_pca_full_10params_df.to_csv('test_pca_full_10params_df.csv')
test

In [ ]:
test.shape

In [ ]:
prediction = catboost_model.predict(test)

In [ ]:
sample = pd.read_csv('Исходники/submit_Sample.csv') # прочитаем шаблон
sample['price'] = prediction # Запишем прогноз
sample['price'] = sample['price'].map(abs)

In [ ]:
sample.to_csv("_submission_2.csv", index = False) # Запишем в файл